# Optimization Test

### Importing libs and data

In [439]:
import pandas as pd
import pyomo.environ as pyo
from pathlib import Path
import os

In [440]:
ROOT = Path(__name__).resolve().parent.parent
FILE = "generic_input_case.xlsx"
FILE_PATH = os.path.join(ROOT, "data", FILE)

In [441]:
pdata = dict()

with pd.ExcelFile(FILE_PATH) as workbook:
    sheets = workbook.sheet_names
    for sheet in sheets:
        pdata[sheet] = pd.read_excel(workbook, sheet_name=sheet)

c:\Users\Vinicius\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


### Data treatment

In [442]:
pdata["HORIZONTE"] = pdata["HORIZONTE"].drop(["CICLO_LENTO"], axis=1)
pdata["FROTA"] = pdata["FROTA"].dropna(axis=1)
pdata["BD_UP"] = pdata["BD_UP"].dropna(axis=1).drop(["DATA_COLHEITA", "IDADE_FLORESTA", "IMA", "RESERVADO"], axis=1)

In [443]:
pdata["period_set"] = pdata["HORIZONTE"]["DIA"].unique()

pdata["logistics_set"] = pdata["FROTA"]["TRANSPORTADOR"].unique()

pdata["up_set"] = pdata["BD_UP"]["UP"].unique()
# pdata["farm_set"] = pdata["BD_UP"]["FAZENDA"].unique()
# Não há necessidade da fazenda. Cada UP é única. Posso somar tudo da UP e fazer por fazenda
# Ou posso fazer somente por UP. Nesse caso, farei por UP para tentar ser mais completo

pdata["factory_set"] = pdata["FABRICA"]["FABRICA"].unique()

In [444]:
logistics_dict = pdata["FROTA"].set_index(["TRANSPORTADOR"]).to_dict()
pdata["min_trucks_param"] = logistics_dict["FROTA_MIN"]
pdata["max_trucks_param"] = logistics_dict["FROTA_MAX"]

grua_dict = pdata["GRUA"].set_index(["TRANSPORTADOR"]).to_dict()
pdata["grua_param"] = grua_dict["QTD_GRUAS"]

up_dict = pdata["BD_UP"].set_index(["UP"]).to_dict()
pdata["density_param"] = up_dict["DB"]
pdata["volume_param"] = up_dict["VOLUME"]
pdata["rsp_param"] = up_dict["RSP"]

factory_dict = pdata["FABRICA"].set_index(["FABRICA", "DIA",]).to_dict()
pdata["min_demand_param"] = factory_dict["DEMANDA_MIN"]
pdata["max_demand_param"] = factory_dict["DEMANDA_MAX"]
pdata["min_rsp_param"] = factory_dict["RSP_MIN"]
pdata["max_rsp_param"] = factory_dict["RSP_MAX"]

route_dict = pdata["ROTA"].set_index(["ORIGEM", "TRANSPORTADOR", "DESTINO"]).to_dict()
pdata["cycle_param"] = route_dict["TEMPO_CICLO"]
route_dict_v2 = pdata["ROTA"].set_index(["TRANSPORTADOR"]).to_dict()
pdata["fitting_box_param"] = route_dict_v2["CAIXA_CARGA"]

In [445]:
pdata["route_possibilities_set"] = route_dict["TEMPO_CICLO"].keys()

### Create Model

In [446]:
model = pyo.ConcreteModel()

### Sets

In [447]:
model.period = pyo.Set(initialize=pdata["period_set"])
model.supplier = pyo.Set(initialize=pdata["logistics_set"])
model.up = pyo.Set(initialize=pdata["up_set"])
model.factory = pyo.Set(initialize=pdata["factory_set"])
model.route_possibilities = pyo.Set(initialize=pdata["route_possibilities_set"])

### Parameters

In [448]:
model.min_trucks = pyo.Param(model.supplier, initialize=pdata["min_trucks_param"])
model.max_trucks = pyo.Param(model.supplier, initialize=pdata["max_trucks_param"])

model.grua = pyo.Param(model.supplier, initialize=pdata["grua_param"])

model.density = pyo.Param(model.up, initialize=pdata["density_param"])
model.volume = pyo.Param(model.up, initialize=pdata["volume_param"])
model.rsp = pyo.Param(model.up, initialize=pdata["rsp_param"])

model.min_demand = pyo.Param(model.factory, model.period, initialize=pdata["min_demand_param"])
model.max_demand = pyo.Param(model.factory, model.period, initialize=pdata["max_demand_param"])
model.min_rsp = pyo.Param(model.factory, model.period, initialize=pdata["min_rsp_param"])
model.max_rsp = pyo.Param(model.factory, model.period, initialize=pdata["max_rsp_param"])

model.cycle = pyo.Param(model.route_possibilities, initialize=pdata["cycle_param"])
model.fitting_box = pyo.Param(model.supplier, initialize=pdata["fitting_box_param"])

### Variables

In [449]:
model.amount_of_trucks = pyo.Var(model.up, model.supplier, model.factory, model.period, domain=pyo.NonNegativeReals)
model.amount_delivery = pyo.Var(model.up, model.supplier, model.factory, model.period, domain=pyo.NonNegativeReals)
model.art_amount_of_trucks = pyo.Var(model.up, model.supplier, model.factory, model.period, domain=pyo.NonNegativeReals)
model.addressed_supplier = pyo.Var(model.up, model.supplier, model.factory, model.period, domain=pyo.Binary)
model.up_started = pyo.Var(model.up, model.supplier, domain=pyo.Binary)
model.up_work = pyo.Var(model.up, model.supplier, model.period, domain=pyo.Binary)

model.db_min = pyo.Var(model.period, domain=pyo.NonNegativeReals)
model.db_max = pyo.Var(model.period, domain=pyo.NonNegativeReals)

### Constraint

In [450]:
def raw_material_minimum_delivery(model, f, p):
    eq = sum(
        model.amount_delivery[u, s, f, p]
        for u in model.up
        for s in model.supplier
        if (u, s, f) in model.route_possibilities
     ) >= model.min_demand[f, p]
    
    return eq

model.raw_material_minimum_delivery = pyo.Constraint(
    model.factory, model.period, rule=raw_material_minimum_delivery
)

In [451]:
def raw_material_maximum_delivery(model, f, p):
    eq = sum(
        model.amount_delivery[u, s, f, p]
        for u in model.up
        for s in model.supplier
        if (u, s, f) in model.route_possibilities
     ) <= model.max_demand[f, p]
    
    return eq

model.raw_material_maximum_delivery = pyo.Constraint(
    model.factory, model.period, rule=raw_material_maximum_delivery
)

In [452]:
# def attend_minimum_rsp(model, f, p):
#     total_volume = sum(model.amount_delivery[u, s, f, p] for u in model.up for s in model.supplier)
#     rsp_value = sum(
#         model.amount_delivery[u, s, f, p] * model.rsp[u]
#         for u in model.up
#         for s in model.supplier
#     )

#     eq = rsp_value >= total_volume * model.min_rsp[f, p]
#     return eq

# model.attend_minimum_rsp = pyo.Constraint(
#     model.factory, model.period, rule=attend_minimum_rsp
# )
def attend_minimum_rsp(model, f, p):
    rsp_value = sum(
        model.addressed_supplier[u, s, f, p] * model.rsp[u]
        for u in model.up
        for s in model.supplier
    )

    qty_rsp_addressed = sum(
        model.addressed_supplier[u, s, f, p]
        for u in model.up
        for s in model.supplier
    )

    eq = rsp_value >= model.min_rsp[f, p] * qty_rsp_addressed
    return eq

model.attend_minimum_rsp = pyo.Constraint(
    model.factory, model.period, rule=attend_minimum_rsp
)

In [453]:
# def attend_maximum_rsp(model, f, p):
#     total_volume = sum(model.amount_delivery[u, s, f, p] for u in model.up for s in model.supplier)
#     rsp_value = sum(
#         model.amount_delivery[u, s, f, p] * model.rsp[u]
#         for u in model.up
#         for s in model.supplier
#     )

#     eq = rsp_value <= total_volume * model.max_rsp[f, p]
#     return eq

# model.attend_maximum_rsp = pyo.Constraint(
#     model.factory, model.period, rule=attend_maximum_rsp
# )

def attend_maximum_rsp(model, f, p):
    rsp_value = sum(
        model.addressed_supplier[u, s, f, p] * model.rsp[u]
        for u in model.up
        for s in model.supplier
    )

    qty_rsp_addressed = sum(
        model.addressed_supplier[u, s, f, p]
        for u in model.up
        for s in model.supplier
    )

    eq = rsp_value <= model.max_rsp[f, p] * qty_rsp_addressed
    return eq

model.attend_maximum_rsp = pyo.Constraint(
    model.factory, model.period, rule=attend_maximum_rsp
)

In [ ]:
def transport_capacity(model, u, s, p):
    M = 10000
    volume_up_day = sum(
        model.amount_delivery[u, s, f, p]
        for f in model.factory
        if (u, s, f) in model.route_possibilities
    )

    cycle = sum(
        model.cycle[u, s, f] * model.fitting_box[s] # 8 Horas de trabalho por dia
        for f in model.factory
        if (u, s, f) in model.route_possibilities
    )

    qty_trucks = sum(
        model.amount_of_trucks[u, s, f, p]
        for f in model.factory
    )

    eq = volume_up_day <=  cycle * qty_trucks #+ M * (1 - sum(model.addressed_supplier[u, s, f, p] for f in model.factory for p in model.period))
    return eq

model.transport_capacity = pyo.Constraint(
    model.up, model.supplier, model.period, rule=transport_capacity
)

In [455]:
def supply_completely(model, u):
    amount_to_send = model.volume[u]

    amount_delivered = sum(
        model.amount_delivery[u, s, f, p]
        # for u in model.up
        for s in model.supplier
        for f in model.factory
        for p in model.period
        if (u, s, f) in model.route_possibilities
    )

    eq = amount_delivered == amount_to_send
    return eq

model.supply_completely = pyo.Constraint(
    model.up, rule=supply_completely
)

In [456]:
def minimum_amount_of_trucks(model, u, s, f, p):
    eq = model.amount_of_trucks[u, s, f, p] >= model.min_trucks[s] * model.addressed_supplier[u, s, f, p]
    return eq

model.minimum_amount_of_trucks = pyo.Constraint(
    model.up, model.supplier, model.factory, model.period, rule=minimum_amount_of_trucks
)

# def minimum_amount_of_trucks(model, s, p):
#     trucks_amount = sum(
#         model.amount_of_trucks[u, s, f, p]
#         for u in model.up
#         for f in model.factory
#     )

#     # addressing = sum(
#     #     model.addressed_supplier[u, s, f, p]
#     #     for u in model.up
#     #     for f in model.factory
#     # )

#     eq = trucks_amount >= model.min_trucks[s]
#     return eq

# model.minimum_amount_of_trucks = pyo.Constraint(
#     model.supplier, model.period, rule=minimum_amount_of_trucks
# )

In [457]:
def maximum_amount_of_trucks(model, u, s, f, p):
    eq = model.amount_of_trucks[u, s, f, p] <= model.max_trucks[s] * model.addressed_supplier[u, s, f, p]
    return eq

model.maximum_amount_of_trucks = pyo.Constraint(
    model.up, model.supplier, model.factory, model.period, rule=maximum_amount_of_trucks
)

# def maximum_amount_of_trucks(model, s, p):
#     trucks_amount = sum(
#         model.amount_of_trucks[u, s, f, p]
#         for u in model.up
#         for f in model.factory
#     )

#     # addressing = sum(
#     #     model.addressed_supplier[u, s, f, p]
#     #     for u in model.up
#     #     for f in model.factory
#     # )

#     eq = trucks_amount <= model.max_trucks[s]
#     return eq

# model.maximum_amount_of_trucks = pyo.Constraint(
#     model.supplier, model.period, rule=maximum_amount_of_trucks
# )

In [458]:

model.maximum_amount_of_trucks.pprint()

maximum_amount_of_trucks : Size=2418, Index=up*supplier*factory*period, Active=True
    Key                              : Lower : Body                                                                                   : Upper : Active
     ('S3AX01', 'Pastori', 'LIM', 1) :  -Inf :   amount_of_trucks[S3AX01,Pastori,LIM,1] - 27*addressed_supplier[S3AX01,Pastori,LIM,1] :   0.0 :   True
     ('S3AX01', 'Pastori', 'LIM', 2) :  -Inf :   amount_of_trucks[S3AX01,Pastori,LIM,2] - 27*addressed_supplier[S3AX01,Pastori,LIM,2] :   0.0 :   True
     ('S3AX01', 'Pastori', 'LIM', 3) :  -Inf :   amount_of_trucks[S3AX01,Pastori,LIM,3] - 27*addressed_supplier[S3AX01,Pastori,LIM,3] :   0.0 :   True
     ('S3AX01', 'Pastori', 'LIM', 4) :  -Inf :   amount_of_trucks[S3AX01,Pastori,LIM,4] - 27*addressed_supplier[S3AX01,Pastori,LIM,4] :   0.0 :   True
     ('S3AX01', 'Pastori', 'LIM', 5) :  -Inf :   amount_of_trucks[S3AX01,Pastori,LIM,5] - 27*addressed_supplier[S3AX01,Pastori,LIM,5] :   0.0 :   True
     ('S3A

In [459]:
# Somente 2 fornecedores operarão do dia devido só termos 2 gruas
def crane_limits(model, s, p):
    eq = sum(
        model.addressed_supplier[u, s, f, p]
        for u in model.up
        for f in model.factory
    ) <= model.grua[s]
    return eq

model.crane_limits = pyo.Constraint(
    model.supplier, model.period, rule=crane_limits
)

In [460]:
def transport_from_up(model, u, s):
    transported_volume = sum(
        model.amount_delivery[u, s, f, p]
        for f in model.factory
        for p in model.period
        if (u, s, f) in model.route_possibilities
    )

    eq = transported_volume == model.volume[u] * model.up_started[u, s]
    return eq

model.transport_from_up = pyo.Constraint(
    model.up, model.supplier, rule=transport_from_up
)

In [461]:
def small_up_attendance(model, u):
    volume_up = model.volume[u]

    if volume_up <= 7000:
        eq = sum(
            # model.up_started[u, s, p]
            model.up_started[u, s]
            for s in model.supplier
        ) <= 1
    else:
        eq = pyo.Constraint.Skip
    return eq

model.small_up_attendance = pyo.Constraint(
    model.up, rule=small_up_attendance
)

In [462]:
def minimum_density(model, u, p):
    M = 20000
    addressing = sum(
        model.addressed_supplier[u, s, f, p]
        for s in model.supplier
        for f in model.factory
        if (u, s, f) in model.route_possibilities
    )

    eq = model.db_min[p] <= model.density[u] + M * addressing
    return eq

model.minimum_density = pyo.Constraint(
    model.up, model.period, rule=minimum_density
)

In [463]:
def maximum_density(model, u, p):
    M = 20000
    addressing = sum(
        model.addressed_supplier[u, s, f, p]
        for s in model.supplier
        for f in model.factory
        if (u, s, f) in model.route_possibilities
    )

    eq = model.db_max[p] >= model.density[u] + M * addressing
    return eq

model.maximum_density = pyo.Constraint(
    model.up, model.period, rule=maximum_density
)

In [464]:
def addressing_supplier(model, u, s, f, p):
    M = 200000
    eq = model.amount_delivery[u, s, f, p] <= M * model.addressed_supplier[u, s, f, p]
    return eq

model.addressing_supplier = pyo.Constraint(
    model.route_possibilities, model.period, rule=addressing_supplier
)

### Objetive

In [465]:
def objetive_function(model):
    # eq2 = sum(
    #     model.amount_delivery[u, s, f, p]
    #     for u in model.up
    #     for s in model.supplier
    #     for f in model.factory
    #     for p in model.period
    #     if (u, s, f) in  model.route_possibilities
    # )
    eq = sum(
        model.db_max[p] - model.db_min[p]
        for p in model.period
    )

    # eq2 =  sum(
    #     model.volume_art[u]
    #     for u in model.up
    # )

    return eq #+ eq2

model.objetive_function = pyo.Objective(
    rule=objetive_function, sense=pyo.minimize
)

In [ ]:
def objetive_function(model):

    eq = sum(
        model.db_max[p] - model.db_min[p]
        for p in model.period
    )

    return eq

model.objetive_function = pyo.Objective(
    rule=objetive_function, sense=pyo.minimize
)

### Solve model

In [466]:
model.write("optimization_test_suzano.lp", io_options={"symbolic_solver_labels":True})

solver = pyo.SolverFactory("appsi_highs")
solver.options['time_limit'] = 300 # segundos
# solver.options['time_limit'] = 300 # segundos
solver.options['mip_rel_gap'] = 0.1
solver.options['mip_abs_gap'] = 1.0


solver.options['presolve'] = 'on'
solver.options['parallel'] = 'on'
solver.solve(model)

load_solution=False and check results.termination_condition and
results.found_feasible_solution() before loading a solution.


{'Problem': [{'Lower bound': 182702.9567771644, 'Upper bound': 622336.2903234606, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 'minimize'}], 'Solver': [{'Status': 'aborted', 'Termination condition': 'maxTimeLimit', 'Termination message': 'TerminationCondition.maxTimeLimit'}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [467]:
# 'Lower bound': 102092.4996334099 Executando em 30 Segundos
# 'Lower bound': xx Executando em 300 Segundos
# 'Lower bound': xx Executando em 600 Segundos

In [468]:
model.amount_delivery.display()

amount_delivery : Size=2418, Index=up*supplier*factory*period
    Key                              : Lower : Value                   : Upper : Fixed : Stale : Domain
     ('S3AX01', 'Pastori', 'LIM', 1) :     0 :                     0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 2) :     0 :                     0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 3) :     0 :                     0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 4) :     0 :      2251.7000000000003 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 5) :     0 :                  3207.6 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 6) :     0 :                     0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 7) :     0 :                     0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 8)

In [469]:
model.amount_of_trucks.display()

amount_of_trucks : Size=2418, Index=up*supplier*factory*period
    Key                              : Lower : Value                  : Upper : Fixed : Stale : Domain
     ('S3AX01', 'Pastori', 'LIM', 1) :     0 :                    0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 2) :     0 :                    0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 3) :     0 :                    0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 4) :     0 :                   27.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 5) :     0 :                   27.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 6) :     0 :                    0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 7) :     0 :                    0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 8) :     

In [470]:
model.up_started.display()

up_started : Size=78, Index=up*supplier
    Key                   : Lower : Value : Upper : Fixed : Stale : Domain
    ('S3AX01', 'Pastori') :     0 :   1.0 :     1 : False : False : Binary
    ('S3AX01', 'Rampazo') :     0 :   0.0 :     1 : False : False : Binary
      ('S3AX01', 'Tover') :     0 :   0.0 :     1 : False : False : Binary
    ('S3AX02', 'Pastori') :     0 :   1.0 :     1 : False : False : Binary
    ('S3AX02', 'Rampazo') :     0 :   0.0 :     1 : False : False : Binary
      ('S3AX02', 'Tover') :     0 :   0.0 :     1 : False : False : Binary
    ('S3AX03', 'Pastori') :     0 :   0.0 :     1 : False : False : Binary
    ('S3AX03', 'Rampazo') :     0 :   1.0 :     1 : False : False : Binary
      ('S3AX03', 'Tover') :     0 :   0.0 :     1 : False : False : Binary
    ('S3AX04', 'Pastori') :     0 :   0.0 :     1 : False : False : Binary
    ('S3AX04', 'Rampazo') :     0 :   1.0 :     1 : False : False : Binary
      ('S3AX04', 'Tover') :     0 :   0.0 :     1 : False : 

In [471]:
model.addressed_supplier.display()

addressed_supplier : Size=2418, Index=up*supplier*factory*period
    Key                              : Lower : Value                   : Upper : Fixed : Stale : Domain
     ('S3AX01', 'Pastori', 'LIM', 1) :     0 :                     0.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 2) :     0 :                     0.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 3) :     0 :                     0.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 4) :     0 :                     1.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 5) :     0 :                     1.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 6) :     0 :                     0.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 7) :     0 :                     0.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 8) :     0 :                     0.0 :     1 : False : False : Binary

In [472]:
model.db_min.display()

db_min : Size=31, Index=period
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
      1 :     0 : 408.37627157534126 :  None : False : False : NonNegativeReals
      2 :     0 :  406.6663018461977 :  None : False : False : NonNegativeReals
      3 :     0 : 408.37627157534126 :  None : False : False : NonNegativeReals
      4 :     0 :  408.3762715753428 :  None : False : False : NonNegativeReals
      5 :     0 : 408.37627157534126 :  None : False : False : NonNegativeReals
      6 :     0 :  406.6663018461977 :  None : False : False : NonNegativeReals
      7 :     0 :  406.6663018461977 :  None : False : False : NonNegativeReals
      8 :     0 :  406.6663018461977 :  None : False : False : NonNegativeReals
      9 :     0 :  406.6663018461977 :  None : False : False : NonNegativeReals
     10 :     0 :  406.6663018461977 :  None : False : False : NonNegativeReals
     11 :     0 :              450.0 :  None : False : False : NonNegativeReals
     12 :     0 :  

In [473]:
model.db_max.display()

db_max : Size=31, Index=period
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
      1 :     0 : 20494.987422568738 :  None : False : False : NonNegativeReals
      2 :     0 : 20485.001740330757 :  None : False : False : NonNegativeReals
      3 :     0 : 20485.001740330757 :  None : False : False : NonNegativeReals
      4 :     0 : 20485.001740330757 :  None : False : False : NonNegativeReals
      5 :     0 :  20476.35931165453 :  None : False : False : NonNegativeReals
      6 :     0 : 20493.203247294947 :  None : False : False : NonNegativeReals
      7 :     0 : 20467.553351316066 :  None : False : False : NonNegativeReals
      8 :     0 : 20494.987422568738 :  None : False : False : NonNegativeReals
      9 :     0 : 20485.001740330757 :  None : False : False : NonNegativeReals
     10 :     0 : 20489.657814115675 :  None : False : False : NonNegativeReals
     11 :     0 : 20493.203247294947 :  None : False : False : NonNegativeReals
     12 :     0 : 2

In [474]:
delivery_df = delivery_df.sort_values(by="DIA", ascending=True).reset_index(drop=True)

In [475]:
delivery_df.query("UP=='S5AK10'").loc[:, "VOLUME"].sum()

np.float64(14000.0)

In [476]:
delivery_df

,UP,FORNECEDOR,FABRICA,DIA,VOLUME
0,S6C298,Tover,LIM,1,0.000000e+00
1,S3AX03,Rampazo,LIM,1,0.000000e+00
2,S5AW10,Pastori,LIM,1,0.000000e+00
3,S5AK10,Rampazo,LIM,1,0.000000e+00
4,S6C297,Tover,LIM,1,0.000000e+00
...,...,...,...,...,...
1235,S6C335,Tover,LIM,31,0.000000e+00
1236,S3AX06,Rampazo,LIM,31,0.000000e+00
1237,S3AX01,Pastori,LIM,31,0.000000e+00
1238,S3AX06,Pastori,LIM,31,0.000000e+00


In [477]:
delivery_df.loc[:, "VOLUME"].sum()

np.float64(200724.0)

In [478]:
trucks_dict = model.amount_of_trucks.extract_values()

trucks_data = [[key[0], key[1], key[2], key[3], value] for key, value in trucks_dict.items()]

trucks_df = pd.DataFrame(
    data=trucks_data,
    columns=["UP", "FORNECEDOR", "FABRICA", "DIA", "QTD_CAMINHOES"]
).dropna()

In [479]:
trucks_df

,UP,FORNECEDOR,FABRICA,DIA,QTD_CAMINHOES
0,S6C298,Pastori,LIM,1,0.0
1,S6C298,Pastori,LIM,2,0.0
2,S6C298,Pastori,LIM,3,0.0
3,S6C298,Pastori,LIM,4,0.0
4,S6C298,Pastori,LIM,5,0.0
...,...,...,...,...,...
2413,S5AK10,Rampazo,LIM,27,0.0
2414,S5AK10,Rampazo,LIM,28,0.0
2415,S5AK10,Rampazo,LIM,29,0.0
2416,S5AK10,Rampazo,LIM,30,0.0


In [480]:
pd.merge(
    left=trucks_df,
    right=delivery_df,
    on=["UP", "FORNECEDOR", "FABRICA", "DIA"],
    how="left"
).sort_values(by=["DIA"]).reset_index(drop=True).to_excel("output_test.xlsx", index=False)